In [105]:
import pickle
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras
import keras.backend as K
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

In [77]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [93]:
img_width, img_height = 150, 150
train_data_dir = './Classification2/data/train'
validation_data_dir = './Classification2/data/validation'
test_dir = validation_data_dir
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [94]:
input_shape = (150, 150, 3)

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',

              optimizer='rmsprop',

              metrics=['accuracy'])

In [95]:
train_datagen = ImageDataGenerator(rescale=1. / 255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(

    train_data_dir,

    target_size=(img_width, img_height),

    batch_size=batch_size,

    class_mode='binary')



validation_generator = test_datagen.flow_from_directory(

    validation_data_dir,

    target_size=(img_width, img_height),

    batch_size=batch_size,

    class_mode='binary')


test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_width, img_height),
        color_mode="rgb",
        shuffle = "false",
        class_mode='binary',
        batch_size=1)

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [98]:
model.fit_generator(

    train_generator,

    steps_per_epoch=nb_train_samples // batch_size,

    epochs=epochs,

    validation_data=validation_generator,
    
    verbose = 1,

    validation_steps=nb_validation_samples // batch_size)





Epoch 1/50
125/125 [==============================] - 223s 2s/step - loss: 0.5391 - acc: 0.7365 - val_loss: 1.0305 - val_acc: 0.6412
Epoch 2/50
125/125 [==============================] - 118s 941ms/step - loss: 0.5265 - acc: 0.7340 - val_loss: 1.2282 - val_acc: 0.6275
Epoch 3/50
125/125 [==============================] - 118s 940ms/step - loss: 0.5109 - acc: 0.7350 - val_loss: 0.7878 - val_acc: 0.6475
Epoch 4/50
125/125 [==============================] - 127s 1s/step - loss: 0.5028 - acc: 0.7615 - val_loss: 1.8512 - val_acc: 0.5975
Epoch 5/50
125/125 [==============================] - 127s 1s/step - loss: 0.5111 - acc: 0.7480 - val_loss: 0.9858 - val_acc: 0.6362
Epoch 6/50
125/125 [==============================] - 123s 988ms/step - loss: 0.5157 - acc: 0.7495 - val_loss: 1.3878 - val_acc: 0.6475
Epoch 7/50
125/125 [==============================] - 123s 986ms/step - loss: 0.5231 - acc: 0.7325 - val_loss: 1.0330 - val_acc: 0.6550
Epoch 8/50
125/125 [==============================] - 118

In [99]:
filenames = test_generator.filenames
nb_samples = len(filenames)
scores = model.evaluate_generator(test_generator,steps = nb_samples)
predictions = model.predict_generator(test_generator,steps = nb_samples)

In [100]:
print (model.metrics_names[0]  + ": "+ str(scores[0]) + "\n"+ model.metrics_names[1] + ": " + str(scores[1]) )

loss: 1.52549578774
acc: 0.625


In [101]:
from sklearn.metrics import confusion_matrix

y_true = np.array([0] * 400 + [1] * 400)
y_pred = predictions > 0.5

confusion_matrix(y_true, y_pred)

array([[284, 116],
       [286, 114]], dtype=int64)

In [102]:
f1s = f1_score(y_true,y_pred, average=None)
print ('F1 Scores:\nMelanoma: %f\nOthers: %f\n'%(f1s[0],f1s[1]))

F1 Scores:
Melanoma: 0.585567
Others: 0.361905



In [108]:
rs = recall_score(y_true,y_pred, average=None)
print ('Recall Score:\nMelanoma: %f\nOthers: %f\n'%(rs[0],rs[1]))

Recall Score:
Melanoma: 0.710000
Others: 0.285000



In [109]:
ps = precision_score(y_true,y_pred, average=None)
print ('Precision Score:\nMelanoma: %f\nOthers: %f\n'%(ps[0],ps[1]))

Precision Score:
Melanoma: 0.498246
Others: 0.495652



In [104]:
model.save_weights('./Classification2/100_epochs_weights.hdf5')

In [110]:
model.load_weights('./Classification2/100_epochs_weights.hdf5')